# <center> Продвинутое машинное обучение
## <center> Домашнее задание № 2

## <center> Хуббатулин Марк. Группа DS2-1

Второе задание — это полноценный проект по анализу данных, начиная от анализа постановки задачи и заканчивая сравнением результатов разных моделей. Задача реальная и серьёзная, хотя тему я выбрал развлекательную: мы будем строить вероятностную рейтинг-систему для спортивного “Что? Где? Когда?” (ЧГК).
Background: в спортивном “Что? Где? Когда?” соревнующиеся команды отвечают на одни и те же вопросы. После минуты обсуждения команды записывают и сдают свои ответы на карточках; побеждает тот, кто ответил на большее число вопросов. Турнир обычно состоит из нескольких десятков вопросов (обычно 36 или 45, иногда 60, больше редко). Часто бывают синхронные турниры, когда на одни и те же вопросы отвечают команды на сотнях игровых площадок по всему миру, т.е. в одном турнире могут играть сотни, а то и тысячи команд. Соответственно, нам нужно:

* построить рейтинг-лист, который способен нетривиально предсказывать результаты будущих турниров;
* при этом, поскольку ЧГК — это хобби, и контрактов тут никаких нет, игроки постоянно переходят из команды в команду, сильный игрок может на один турнир сесть поиграть за другую команду и т.д.; поэтому единицей рейтинг-листа должна быть не команда, а отдельный игрок;
* а что сильно упрощает задачу и переводит её в область домашних заданий на EM-алгоритм — это характер данных: начиная с какого-то момента, в базу результатов начали вносить все повопросные результаты команд, т.е. в данных будут записи вида “какая команда на какой вопрос правильно ответила”.


### Оглавление <a name = 'outline'></a>
* [Задание№1](#part1) 
* [Задание№2](#part2) 
* [Задание№3](#part3) 
* [Задание№4](#part4) 
* [Задание№5](#part5) 
* [Задание№6](#part6) 
* [Задание№7](#part7) 

In [1]:
import json
import gc
import requests
from tqdm import tqdm
from datetime import datetime 
from collections import defaultdict
from itertools import chain

import numpy as np
import pandas as pd
import pickle

from scipy.special import logit, expit
from scipy.stats import spearmanr, kendalltau
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LogisticRegression, LinearRegression

import matplotlib.pyplot as plt
import seaborn as sns

plt.rcParams["figure.figsize"] = (9, 7)

tqdm.pandas()
pd.options.mode.chained_assignment = None 
PREPROCESSING = False
TRAIN = False

PATH = './data/chgk/'

/home/markh/anaconda3/lib/python3.8/site-packages/tqdm/std.py:697: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


> 1. <a name = 'part1'></a>  Прочитайте и проанализируйте данные, выберите турниры, в которых есть данные о составах команд и повопросных результатах (поле mask в results.pkl). Для унификации предлагаю:
взять в тренировочный набор турниры с dateStart из 2019 года; 
в тестовый — турниры с dateStart из 2020 года.

In [2]:
with open(PATH + 'row/players.pkl', 'rb') as fin:
    players = pickle.load(fin)

In [3]:
def get_current_rating(player_id): 
    try:
        rating = requests.get(f"https://rating.chgk.info/api/players/{player_id}/rating/last").json()['rating_position']
    except:
        rating = 0
    return rating

In [4]:
players = pd.DataFrame(players).T
players.head(5)

,id,name,patronymic,surname
1,1,Алексей,None,Абабилов
10,10,Игорь,,Абалов
11,11,Наталья,Юрьевна,Абалымова
12,12,Артур,Евгеньевич,Абальян
13,13,Эрик,Евгеньевич,Абальян


In [5]:
print(f'Количество игроков {len(players)}')

Количество игроков 204063


In [6]:
if PREPROCESSING:
    with open(PATH + 'row/tournaments.pkl', 'rb') as f:
        tournaments = pickle.load(f)

    with open(PATH + 'row/results.pkl', 'rb') as file:
        result = pickle.load(file)

In [7]:
def train_test_split(tournaments, result):
    tournaments = pd.DataFrame(tournaments).T
    tournaments.set_index('id', inplace=True)
    tournaments['dateStart'] = pd.to_datetime(tournaments['dateStart'], utc=True)
    tournaments['dateEnd'] = pd.to_datetime(tournaments['dateEnd'], utc=True)
    tournaments_2019 = tournaments[tournaments['dateStart'].dt.year == 2019]
    tournaments_2020 = tournaments[tournaments['dateStart'].dt.year == 2020]
    
    result_filtered = {}
    for key, value in result.items():
        if len(value) > 0:
            try:
                if (value[0]['mask'] is not None) and (value[0]['teamMembers'] != []):
                    result_filtered[key] = value
            except KeyError:
                pass 
            
    train_index = set(list(set(result_filtered.keys()) & set(tournaments_2019.index)))
    test_index = set(list(set(result_filtered.keys()) & set(tournaments_2020.index)))
    
    train = {}
    test = {}

    for index in train_index:
        train[index] = result_filtered[index]

    for index in test_index:
        test[index] = result_filtered[index]
        
    return train, test

In [8]:
if PREPROCESSING:
    train, test = train_test_split(tournaments, result)

    del result
    gc.collect();

In [9]:
if PREPROCESSING:    
    with open(PATH + 'filtred/train.json', 'w') as fout:
        json.dump(train, fout)
    with open(PATH + 'filtred/test.json', 'w') as fout:
        json.dump(test, fout)
        
with open(PATH + 'filtred/train.json', 'rb') as fin:
    train = json.load(fin)
with open(PATH + 'filtred/test.json', 'rb') as fin:
    test = json.load(fin)

In [10]:
print(f"Количество турниров в обучающей выборке - {len(train)}")
print(f"Количество турниров в тестовой выборке - {len(test)} \n")

Количество турниров в обучающей выборке - 676
Количество турниров в тестовой выборке - 171 



In [11]:
def make_dataset(train, test):
    data = []
    for idx, tournament in train.items():
        for team in tournament:
            for member in team['teamMembers']:
                player = member['player']
                data.append({'tournament_id': idx,
                            'year': 2019,
                            'team_id': team['team']['id'],
                            'mask': team['mask'],
                            'answered': team['questionsTotal'],
                            'player_id': player['id'],
                            'position': team['position'],
                            }
                        )

    for idx, tournament in test.items():
        for team in tournament:
            for member in team['teamMembers']:
                player = member['player']
                data.append({'tournament_id': idx,
                            'year': 2020,
                            'team_id': team['team']['id'],
                            'mask': team['mask'],
                            'answered': team['questionsTotal'],
                            'player_id': player['id'],
                            'position': team['position'],
                            }
                        )   
    df = pd.DataFrame.from_records(data)
    df['question_cnt'] = df['mask'].str.len()
    df.dropna(subset=['mask'], inplace=True)
    df['mask'] = df['mask'].str.replace('X', "0")
    df['mask'] = df['mask'].str.replace('?', "0")

    questions = []
    for l in df['question_cnt']:
        questions.extend(np.arange(1, l + 1))
    
    filtred_df = pd.DataFrame({
                                'tournament_id': np.repeat(df['tournament_id'], df['question_cnt']),
                                'year': np.repeat(df['year'], df['question_cnt']),
                                'team_id': np.repeat(df['team_id'], df['question_cnt']),
                                'position': np.repeat(df['position'], df['question_cnt']),
                                'player_id': np.repeat(df['player_id'], df['question_cnt']),
                                'question_id': questions,
                                'answered_status': list(chain.from_iterable(df['mask']))

                                }).reset_index(drop=True)
    filtred_df['question_id'] = filtred_df['question_id'].astype(int)
    filtred_df['question_id'] = filtred_df['tournament_id'].astype(str) + '_' + filtred_df['question_id'].astype(str)
    return filtred_df

In [12]:
if PREPROCESSING:    
    filtred_df = make_dataset(train, test)
    
    train = filtred_df[filtred_df['year'] == 2019]
    test = filtred_df[filtred_df['year'] == 2020]
    
    train.to_csv('data/chgk/filtred/train.csv')
    test.to_csv('data/chgk/filtred/test.csv')
    
    filtred_df.head(5)

In [13]:
train = pd.read_csv('data/chgk/filtred/train.csv', index_col=0)
test = pd.read_csv('data/chgk/filtred/test.csv', index_col=0)

/home/markh/anaconda3/lib/python3.8/site-packages/numpy/lib/arraysetops.py:583: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [14]:
if PREPROCESSING:    
    del filtred_df
    
gc.collect();

[Вернуться к оглавлению](#outline)

> 2. <a name = 'part2'></a> Постройте baseline-модель на основе линейной или логистической регрессии, которая будет обучать рейтинг-лист игроков. Замечания и подсказки:
> * повопросные результаты — это фактически результаты броска монетки, и их предсказание скорее всего имеет отношение к бинарной классификации;
> * в разных турнирах вопросы совсем разного уровня сложности, поэтому модель должна это учитывать; скорее всего, модель должна будет явно обучать не только силу каждого игрока, но и сложность каждого вопроса;
> * для baseline-модели можно забыть о командах и считать, что повопросные результаты команды просто относятся к каждому из её игроков.


Попробуем обучить Логистическую регрессию. Как признаки используем закодированных игроков и вопросы, тем самым будем учитывать силу игроков и сложность вопросов. Будем пытаться предсказать ответит игрок на вопрос или нет $p(a=1|player_i, question_i)$

Полученные веса можно интерпретировать как уровень игрока. Тогда рейтинг игрока будет равен $r(a=1|player_i) = \sigma(w_0 + w_{player})$.

In [15]:
player_question_encoder = OneHotEncoder(handle_unknown='ignore')
X_train = player_question_encoder.fit_transform(train[['player_id', 'question_id']])
X_test = player_question_encoder.transform(test[['player_id', 'question_id']])

y_train = train['answered_status']
y_test = test['answered_status']

players_encoder_cat = player_question_encoder.categories_[0]
question_encoder_cat = player_question_encoder.categories_[1]

In [16]:
if TRAIN:
    base_logreg = LogisticRegression(solver='saga')
    base_logreg.fit(X_train, y_train)

In [17]:
pkl_filename = PATH + "assets/log_reg_base_model.pkl"

if TRAIN:
    with open(pkl_filename, 'wb') as file:
        pickle.dump(base_logreg, file)

with open(pkl_filename, 'rb') as file:
    base_logreg = pickle.load(file)

In [18]:
w0 = base_logreg.intercept_
base_weights_players = dict(zip(players_encoder_cat, base_logreg.coef_[0][:len(players_encoder_cat)]))
base_weights_question = dict(zip(question_encoder_cat, base_logreg.coef_[0][len(players_encoder_cat):]))

In [19]:
base_logreg_rating = train[['player_id']].drop_duplicates().reset_index(drop=True)
base_logreg_rating['logreg_weight'] = base_logreg_rating['player_id'].map(base_weights_players)
base_logreg_rating['r'] = expit(w0 + base_logreg_rating['logreg_weight'])
base_logreg_rating['predicted_rank'] = base_logreg_rating['r'].rank(method='dense', ascending=False)

In [20]:
base_logreg_rating = base_logreg_rating.merge(players, left_on='player_id', right_index=True)
base_logreg_rating = base_logreg_rating.merge(train.groupby('player_id')['tournament_id'].nunique().rename('tournament_cnt'),
                                    left_on='player_id', right_index=True )
base_logreg_rating

,player_id,logreg_weight,r,predicted_rank,id,name,patronymic,surname,tournament_cnt
0,27469,2.209247,0.637627,1688.0,27469,Денис,Николаевич,Рыбачук,30
1,57286,1.920619,0.568676,3200.0,57286,Кирилл,Владимирович,Климович,51
2,155103,1.872331,0.556795,3494.0,155103,Виталий,Николаевич,Ковалёв,49
3,41104,1.920941,0.568755,3197.0,41104,Александр,Николаевич,Кухарчук,49
4,57288,1.910979,0.566310,3243.0,57288,Михаил,Валерьевич,Мурасин,42
...,...,...,...,...,...,...,...,...,...
59265,216849,-0.501647,0.104723,38986.0,216849,Николай,Игоревич,Красковский,1
59266,216850,-0.501649,0.104723,38988.0,216850,Дмитрий,Сергеевич,Кузьмин,1
59267,216851,-0.501653,0.104723,38990.0,216851,Жанна,Юрьевна,Климёнок,1
59268,216852,-0.501648,0.104723,38987.0,216852,Дарья,Вячеславовна,Павлова,1


In [21]:
player_rating_top_50_base_logreg = base_logreg_rating.sort_values(by='predicted_rank').head(50)
player_rating_top_50_base_logreg['current_rating'] = player_rating_top_50_base_logreg.apply(lambda x: get_current_rating(x['player_id']), axis=1).astype(int)

In [22]:
player_rating_top_50_base_logreg

,player_id,logreg_weight,r,predicted_rank,id,name,patronymic,surname,tournament_cnt,current_rating
25142,27403,4.102865,0.921195,1.0,27403,Максим,Михайлович,Руссо,59,5
18447,4270,3.976155,0.911492,2.0,4270,Александра,Владимировна,Брутер,72,6
18448,28751,3.949148,0.909289,3.0,28751,Иван,Николаевич,Семушин,100,3
17887,30152,3.773089,0.893683,4.0,30152,Артём,Сергеевич,Сорожкин,130,1
14444,27822,3.768901,0.893284,5.0,27822,Михаил,Владимирович,Савченков,85,2
14446,30270,3.764555,0.892869,6.0,30270,Сергей,Леонидович,Спешков,93,4
14442,20691,3.622842,0.878540,7.0,20691,Станислав,Григорьевич,Мереминский,39,38
21848,18036,3.619908,0.878226,8.0,18036,Михаил,Ильич,Левандовский,34,8
21849,26089,3.574006,0.873232,9.0,26089,Ирина,Сергеевна,Прокофьева,27,65
424,22799,3.568669,0.872640,10.0,22799,Сергей,Игоревич,Николенко,51,10


In [23]:
len_actial = len(player_rating_top_50_base_logreg[player_rating_top_50_base_logreg['current_rating'] <= 50])
print(f"Количество игроков, которые действительно в топ 50: {len_actial}")

Количество игроков, которые действительно в топ 50: 23


Впринице получилось очень даже неплохо. Видно, что в топ рейтинга попадают игроки с маленьким количеством участий в турнирах (даже есть по 1ому участию)

In [24]:
base_logreg_rating.to_csv('data/chgk/rating/base_logreg_rating.csv')

In [25]:
del base_logreg_rating
del player_rating_top_50_base_logreg

gc.collect();

[Вернуться к оглавлению](#outline)

> 3. <a name = 'part3'></a> Качество рейтинг-системы оценивается качеством предсказаний результатов турниров. Но сами повопросные результаты наши модели предсказывать вряд ли смогут, ведь неизвестно, насколько сложными окажутся вопросы в будущих турнирах; да и не нужны эти предсказания сами по себе. Поэтому:
> * предложите способ предсказать результаты нового турнира с известными составами, но неизвестными вопросами, в виде ранжирования команд;
> * в качестве метрики качества на тестовом наборе давайте считать ранговые корреляции Спирмена и Кендалла (их можно взять в пакете scipy) между реальным ранжированием в результатах турнира и предсказанным моделью, усреднённые по тестовому множеству турниров.

Так как нам известны составы и у нас обучена модель на пресказание вероятности ответа на вопрос игроком, то представим вероятность команды ответить на вопрос как  вероятность того, что хотя бы один из игроков ответитит на вопрос:
$$p(a=1|team) = 1 - \prod_{player \in team} (1 - p(a=1|player))$$

In [26]:
def get_spear_kendal_correlation(p):

    pred_ranks = test[['tournament_id', 'team_id']]
    pred_ranks.loc[:, '1-p'] = 1 - p
    pred_ranks = pred_ranks.groupby(['tournament_id', 'team_id']).prod().reset_index()
    pred_ranks['predicted_position'] = pred_ranks.groupby('tournament_id')['1-p'].rank('dense')
    
    true_ranks = test[['tournament_id', 'team_id', 'position']].drop_duplicates()
    ranks = pd.merge(pred_ranks, true_ranks, on=['tournament_id', 'team_id'])

    spear_corr = []
    kend_corr = []
    for tournament in ranks['tournament_id'].unique():
        i = ranks[ranks['tournament_id'] == tournament]
        if len(i) > 1:
            spear_corr.append(spearmanr(i['position'], i['predicted_position'])[0])
            kend_corr.append(kendalltau(i['position'], i['predicted_position'])[0])
    return np.mean(spear_corr), np.mean(kend_corr)

In [27]:
p = base_logreg.predict_proba(X_test)[:, 1]

In [28]:
spear_corr, kendal_corr = get_spear_kendal_correlation(p)
print(f'Корреляции Спирмена:{spear_corr} --- Корреляции Кендалла:{kendal_corr}')

Корреляции Спирмена:0.7844663756895934 --- Корреляции Кендалла:0.6264022596624789


Корреляции в пределах нормы

[Вернуться к оглавлению](#outline)

4. <a name = 'part4'></a> Теперь главное: ЧГК — это всё-таки командная игра. Поэтому:
* предложите способ учитывать то, что на вопрос отвечают сразу несколько игроков; скорее всего, понадобятся скрытые переменные; не стесняйтесь делать упрощающие предположения, но теперь переменные “игрок X ответил на вопрос Y” при условии данных должны стать зависимыми для игроков одной и той же команды;
* разработайте EM-схему для обучения этой модели, реализуйте её в коде;
* обучите несколько итераций, убедитесь, что целевые метрики со временем растут (скорее всего, ненамного, но расти должны), выберите лучшую модель, используя целевые метрики.

Используем скрытую переменную $z_i$. Она будет показывать вероятность ответить на вопрос игроком при условии, что он состоит в определенной команде $z_i = p(x_i|team)$

Инициализируем веса линейной регресии изначальными вероятностями ответа игроков внезависимости от команды

* E-шаг: замораживаем веса, вычисляем $z_{i}$ 
Предпологаем о ответе команды как в предыдущем пункте - команда отвечает правильно, если хотя бы один из игроков знает ответ, а если отвечает неправильно - то никто из  игроков не знает ответа. Т.е 

$E[z_i]=0$, если $y = 0$

$E[z_i]= \frac{p(z_i)}{1 - \prod_{z \in team} (1 - p(z_i))}$, если $y = 1$


* М-шаг: фиксируем $z_i$ обучаем линейную регресиию на $logit(p(z_i))$
  Регрессия вернет нам $p(x_i)$. $\sigma(p(x_i))$ Снова подается на E-шаг

In [29]:
train_em = train.copy(deep=True)
train_em['p'] = base_logreg.predict_proba(X_train)[:, 1]

lg_model = LinearRegression()

In [30]:
class EM():
    
    def __init__(self, model):
        self.model = model
        self.best_model = model
        self.best_spear = 0
        self.best_kendal = 0
        
    def fit(self, train, X_train, y_train):
        print("обучение EM")
        for step in tqdm(range(4)):
            #E Step
            train['1-p'] = 1 - train['p']
            teams = 1 - train.groupby(['tournament_id', 'team_id', 'question_id'])['1-p'].prod()
            train = train.merge(teams.rename('team_p'), left_on=['tournament_id', 'team_id', 'question_id'], right_index=True)
            train['z'] = train['p'] / train['team_p']
            train['z'] = np.where(y_train == 0, 0, train['z'])
            train['z'] = np.clip(train['z'], 1e-6, 1 - 1e-6)
            #M Step
            self.model.fit(X_train, logit(train['z']))
            train['p'] = expit(self.model.predict(X_train))
            train = train.drop('team_p',1)
            
            
            test_y = expit(self.model.predict(X_test))
            spear_corr, kendal_corr = get_spear_kendal_correlation(test_y)
            if spear_corr > self.best_spear and kendal_corr > self.best_kendal:
                self.best_model = self.model
            print(f'Корреляции Спирмена:{spear_corr} --- Корреляции Кендалла:{kendal_corr}')

In [31]:
em_model = EM(lg_model)
em_model.fit(train_em, X_train, y_train)
best_model = em_model.best_model

  0%|          | 0/4 [00:00<?, ?it/s]

обучение EM


 25%|██▌       | 1/4 [04:45<14:15, 285.32s/it]

Корреляции Спирмена:0.8038122245498545 --- Корреляции Кендалла:0.6513868736750387


 50%|█████     | 2/4 [09:34<09:32, 286.39s/it]

Корреляции Спирмена:0.8060845755103185 --- Корреляции Кендалла:0.64866800484976


 75%|███████▌  | 3/4 [14:22<04:47, 287.09s/it]

Корреляции Спирмена:0.806679973491553 --- Корреляции Кендалла:0.6490894790617016


100%|██████████| 4/4 [19:13<00:00, 288.29s/it]

Корреляции Спирмена:0.8011819910992812 --- Корреляции Кендалла:0.6438213456316343


In [32]:
pkl_filename = PATH + "assets/em_model.pkl"

with open(pkl_filename, 'wb') as file:
    pickle.dump(best_model, file)

with open(pkl_filename, 'rb') as file:
    best_model = pickle.load(file)

Относительно baseline модели корреляции выросли на 2%. Целевые метрики росли до определенного момента, затем остановили свой рост

[Вернуться к оглавлению](#outline)

> 5. <a name = 'part5'></a> А что там с вопросами? Постройте “рейтинг-лист” турниров по сложности вопросов. Соответствует ли он интуиции (например, на чемпионате мира в целом должны быть сложные вопросы, а на турнирах для школьников — простые)? Если будет интересно: постройте топ сложных и простых вопросов со ссылками на конкретные записи в базе вопросов ЧГК (это чисто техническое дело, тут никакого ML нету).

Также воспользуемся весами, только в этот раз для вопросов

In [33]:
question_weights_em = dict(zip(question_encoder_cat, best_model.coef_[len(players_encoder_cat):]))

In [34]:
with open(PATH + 'row/tournaments.pkl', 'rb') as f:
    tournaments = pickle.load(f)
tournaments = pd.DataFrame(tournaments).T
tournaments.set_index('id', inplace=True)
tournaments['dateStart'] = pd.to_datetime(tournaments['dateStart'], utc=True)
tournaments['dateEnd'] = pd.to_datetime(tournaments['dateEnd'], utc=True)

In [35]:
em_rating = train[['tournament_id', 'question_id']].drop_duplicates()
em_rating['question_weight'] = em_rating['question_id'].map(question_weights_em)
em_rating = em_rating.groupby('tournament_id')['question_weight'].mean().reset_index()
em_rating = em_rating.merge(tournaments[['name']], left_on='tournament_id', right_index=True)
em_rating = em_rating.sort_values(by='question_weight', ascending=True)

In [36]:
em_rating['name'].head(25)

664    Чемпионат Санкт-Петербурга. Первая лига
542                                Угрюмый Ёрш
368                 Синхрон высшей лиги Москвы
43                Первенство правого полушария
553           Чемпионат Мира. Этап 2. Группа В
178                           Чемпионат России
665    Чемпионат Санкт-Петербурга. Высшая лига
554            Чемпионат Мира. Этап 2 Группа С
552           Чемпионат Мира. Этап 2. Группа А
557           Чемпионат Мира. Этап 3. Группа В
640                         Воображаемый музей
376                           Знание – Сила VI
561            Чемпионат Мира. Финал. Группа С
26                           Ускользающая сова
556           Чемпионат Мира. Этап 3. Группа А
283                           Записки охотника
558           Чемпионат Мира. Этап 3. Группа С
224    Чемпионат Минска. Лига А. Тур четвёртый
67                Мемориал Дмитрия Коноваленко
639       Чемпионат Минска. Лига А. Тур второй
549           Чемпионат Мира. Этап 1. Группа А
405          

In [37]:
em_rating['name'].tail(25)

240                                    Кубок Тышкевичей
468                   Шестой киевский марафон. Асинхрон
9               (а)Синхрон-lite. Лига старта. Эпизод IV
447                Кубок княгини Ольги среди школьников
6                     Школьный Синхрон-lite. Выпуск 2.3
71                 Парный асинхронный турнир ChGK is...
378                   Школьный Синхрон-lite. Выпуск 3.1
674                                       Школьная лига
385              (а)Синхрон-lite. Лига старта. Эпизод X
382                   Школьный Синхрон-lite. Выпуск 3.3
295                Межфакультетский кубок МГУ. Отбор №4
564                              Школьная лига. II тур.
7              (а)Синхрон-lite. Лига старта. Эпизод III
547                               Школьная лига. I тур.
10                    Школьный Синхрон-lite. Выпуск 2.5
386                   Школьный Синхрон-lite. Выпуск 3.5
379            (а)Синхрон-lite. Лига старта. Эпизод VII
383             (а)Синхрон-lite. Лига старта. Эп

Выглядит справедливо - Мировые чемпионаты в топе, школьные и сихрон-lite в самом низу

[Вернуться к оглавлению](#outline)

> 6. <a name = 'part6'></a>  Бонус: постройте топ игроков по предсказанной вашей моделью силе игры, а рядом с именами игроков напишите общее число вопросов, которое они сыграли. Скорее всего, вы увидите, что топ занят игроками, которые сыграли совсем мало вопросов, около 100 или даже меньше; если вы поищете их в официальном рейтинге ЧГК, вы увидите, что это какие-то непонятные ноунеймы. В baseline-модели, скорее всего, такой эффект будет гораздо слабее.
Это естественное свойство модели: за счёт EM-схемы влияние 1-2 удачно сыгранных турниров будет только усиливаться, потому что неудачных турниров, чтобы его компенсировать, у этих игроков нет. Более того, это не мешает метрикам качества, потому что если эти игроки сыграли всего 1-2 турнира в 2019-м, скорее всего они ничего или очень мало сыграли и в 2020, и их рейтинги никак не влияют на качество тестовых предсказаний. Но для реального рейтинга такое свойство, конечно, было бы крайне нежелательным. Давайте попробуем его исправить:
> * сначала жёстко: выберите разумную отсечку по числу вопросов, учитывая, что в одном турнире их обычно 30-50;
> * можно ли просто выбросить игроков, которые мало играли, и переобучить модель? почему? предложите, как нужно изменить модель, чтобы не учитывать слишком мало сыгравших, и переобучите модель;
> * но всё-таки это не слишком хорошее решение: если выбрать маленькую отсечку, будут ноунеймы в топе, а если большую, то получится, что у нового игрока слишком долго не будет рейтинга; скорее всего, никакой “золотой середины” тут не получится;
> * предложите более концептуальное решение для топа игроков в рейтинг-листе; если получится, реализуйте его на практике (за это уж точно будут серьёзные бонусные баллы).


Снова построим рейтинг, но уже по весами EM модели

In [38]:
player_weights_em = dict(zip(players_encoder_cat, best_model.coef_[:len(players_encoder_cat)]))

In [39]:
em_player_rating = train[['player_id']].drop_duplicates().reset_index(drop=True)
em_player_rating['logreg_weight'] = em_player_rating['player_id'].map(player_weights_em)
em_player_rating['r'] = expit(w0 + em_player_rating['logreg_weight'])
em_player_rating['predicted_rank'] = em_player_rating['r'].rank(method='dense', ascending=False)

In [40]:
em_player_rating = em_player_rating.merge(players, left_on='player_id', right_index=True)
em_player_rating = em_player_rating.merge(train.groupby('player_id')['tournament_id'].nunique().rename('tournament_cnt'),
                                    left_on='player_id', right_index=True )
em_player_rating = em_player_rating.merge(train.groupby('player_id')['question_id'].count().rename('question_cnt'),
                                    left_on='player_id', right_index=True)

In [41]:
player_rating_top_50_em = em_player_rating.sort_values(by='predicted_rank').head(50)
player_rating_top_50_em['current_rating'] = player_rating_top_50_em.apply(lambda x: get_current_rating(x['player_id']), axis=1).astype(int)

In [42]:
player_rating_top_50_em

,player_id,logreg_weight,r,predicted_rank,id,name,patronymic,surname,tournament_cnt,question_cnt,current_rating
3801,212663,12.642959,0.999983,1.0,212663,Майя,Александровна,Губина,3,108,0
51110,36844,11.953142,0.999967,2.0,36844,Павел,Константинович,Щербина,1,36,13772
44427,22474,11.831651,0.999962,3.0,22474,Илья,Сергеевич,Немец,2,75,4471
48876,136300,8.905571,0.999298,4.0,136300,Александра,Петровна,Буйная,1,36,0
50046,206275,8.686797,0.999127,5.0,206275,Руслан,Васильевич,Гайфутдинов,3,108,8856
18861,168352,8.421627,0.998862,6.0,168352,Михаил,Сергеевич,Григорьев,4,156,4354
48875,202410,7.851540,0.997990,7.0,202410,Валентина,,Подюкова,1,36,0
18447,4270,7.389986,0.996814,8.0,4270,Александра,Владимировна,Брутер,72,3026,6
25142,27403,7.372466,0.996758,9.0,27403,Максим,Михайлович,Руссо,59,2474,5
50150,211291,7.295842,0.996501,10.0,211291,Валерий,Терентьевич,Галкин,2,72,25496


In [43]:
len_actial = len(player_rating_top_50_em[(player_rating_top_50_em['current_rating'] <= 50) & (player_rating_top_50_em['current_rating'] != 0)])
print(f"Количество игроков, которые действительно в топ 50: {len_actial}")

Количество игроков, которые действительно в топ 50: 14


Да, появилось больше нонеймов. Количество людей, действительно в топе снизилось по сравнению с baseline моделью.

Попробуем сделать жесткую отсечку как по количеству вопросов, так и по количеству турниров

In [44]:
more_50_question = em_player_rating[(em_player_rating['question_cnt'] > 100) & (em_player_rating['tournament_cnt'] > 3)]

In [45]:
player_rating_top_50_em_more_50q = more_50_question.sort_values(by='predicted_rank').head(50)
player_rating_top_50_em_more_50q['current_rating'] = player_rating_top_50_em_more_50q.apply(lambda x: get_current_rating(x['player_id']), axis=1).astype(int)

In [46]:
player_rating_top_50_em_more_50q

,player_id,logreg_weight,r,predicted_rank,id,name,patronymic,surname,tournament_cnt,question_cnt,current_rating
18861,168352,8.421627,0.998862,6.0,168352,Михаил,Сергеевич,Григорьев,4,156,4354
18447,4270,7.389986,0.996814,8.0,4270,Александра,Владимировна,Брутер,72,3026,6
25142,27403,7.372466,0.996758,9.0,27403,Максим,Михайлович,Руссо,59,2474,5
18448,28751,7.287886,0.996473,11.0,28751,Иван,Николаевич,Семушин,100,4118,3
14444,27822,6.810597,0.994327,16.0,27822,Михаил,Владимирович,Савченков,85,3677,2
17887,30152,6.804140,0.994291,17.0,30152,Артём,Сергеевич,Сорожкин,130,5254,1
23978,19915,6.715314,0.993764,18.0,19915,Александр,Валерьевич,Марков,74,3106,51
14446,30270,6.671248,0.993485,19.0,30270,Сергей,Леонидович,Спешков,93,4190,4
13532,74382,6.607507,0.993059,20.0,74382,Михаил,Андреевич,Новосёлов,74,3280,50
14442,20691,6.426200,0.991691,27.0,20691,Станислав,Григорьевич,Мереминский,39,1739,38


In [47]:
len_actial = len(player_rating_top_50_em_more_50q[(player_rating_top_50_em_more_50q['current_rating'] <= 50) & (player_rating_top_50_em_more_50q['current_rating'] != 0)])
print(f"Количество игроков, которые действительно в топ 50: {len_actial}")

Количество игроков, которые действительно в топ 50: 21


Ситуация стала лучше, но всё равно уступает пока даже бэйзлайну. 

[Вернуться к оглавлению](#outline)

> 7. <a name = 'part7'></a> Бонус: игроки со временем учатся играть лучше (а иногда бывает и наоборот). А в нашей модели получается, что первые неудачные турниры новичка будут тянуть его рейтинг вниз всю жизнь — это нехорошо, рейтинг должен быть достаточно гибким и иметь возможность меняться даже у игроков, отыгравших сотни турниров. Давайте попробуем этого добиться:
> * если хватит вычислительных ресурсов, сначала сделайте baseline совсем без таких схем, обучив рейтинги на всех турнирах с повопросными результатами, а не только на турнирах 2019 года; улучшилось ли качество предсказаний на 2020?
> * одну схему со временем мы уже использовали: брали для обучения только последний год турниров; примерно так делают, например, в теннисной чемпионской гонке; у этой схемы есть свои преимущества, но есть и недостатки (например, достаточно мало играть год, чтобы полностью пропасть из рейтинга);
> * предложите варианты базовой модели или алгоритма её обучения, которые могли бы реализовать изменения рейтинга со временем; если получится, реализуйте их на практике, проверьте, улучшатся ли предсказания на 2020.


К сожалению Оперативки для загрузки не хватает:(. Опишу саму схему.
                                                
Обучаем baseline как и во втором пункте, только группируем ещё и по годам и взвешивает рейтинг игрока по годам (чем дальше был верный ответ на турнире, тем меньший вес он получает). Веса кодируем от 0 до 1 между годами выборки

[Вернуться к оглавлению](#outline)